In [50]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments
from datasets import load_dataset, load_metric, Dataset

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
import pickle
import pandas as pd
from datasets import Value
from torch.utils.data import DataLoader
import argparse
import os
from pymongo import MongoClient
import IPython
from tqdm import tqdm

In [30]:
client = MongoClient()
db_metoo_tweets = client["jason_twitter"]
metoo_tweets = db_metoo_tweets.tweets

In [31]:
tweet_cursor = metoo_tweets.find({"condemnation_prediction":1})

In [32]:
user_ids = set()

In [33]:
for tweet in tqdm(tweet_cursor):
    user_ids.add(tweet["user"]["id"])


8257732it [23:54, 5757.79it/s]


In [58]:
len(user_ids)

1405212

In [103]:
pf_df = pd.read_csv("../../data/ideology-data/followers_list.csv")

In [104]:
pf_df = pf_df.rename(columns={"Unnamed: 0": "account"})

In [105]:
pf_df

,account,followers_list
0,1002630999052865536,"['1477015231444434952', '1462898232367931410',..."
1,1004891731,"['1595422545167147010', '1631293374195175424',..."
2,10168082,"['1648278229558636545', '37696245', '123178873..."
3,1017500185356853248,"['1112153645129367552', '1609366275041513476',..."
4,1017819745880543238,"['3039909098', '4869664833', '1063922766', '30..."
...,...,...
564,976969338,"['3755222237', '1505081953078317057', '3976808..."
565,983348251972816896,"['537330880', '1502751179251539970', '73837842..."
566,993153006,"['1066374611754016768', '837796797514919936', ..."
567,995193054,"['1600781442409287682', '1647355610533330944',..."


In [109]:
pf_df["followers_count"] = pf_df.followers_list.apply(lambda x: len(ast.literal_eval(x)))

In [110]:
pf_df

,account,followers_list,followers_count
0,1002630999052865536,"['1477015231444434952', '1462898232367931410',...",5000
1,1004891731,"['1595422545167147010', '1631293374195175424',...",5000
2,10168082,"['1648278229558636545', '37696245', '123178873...",5000
3,1017500185356853248,"['1112153645129367552', '1609366275041513476',...",5000
4,1017819745880543238,"['3039909098', '4869664833', '1063922766', '30...",5000
...,...,...,...
564,976969338,"['3755222237', '1505081953078317057', '3976808...",5000
565,983348251972816896,"['537330880', '1502751179251539970', '73837842...",5000
566,993153006,"['1066374611754016768', '837796797514919936', ...",5000
567,995193054,"['1600781442409287682', '1647355610533330944',...",5000


In [86]:

new_rows = []
for idx, row in tqdm(pf_df.iterrows()):
    new_rows.extend({"account" : row["account"]  , "id_str": u} for u in ast.literal_eval(row["followers_list"]))
    
pf_df = pd.DataFrame(new_rows)
pf_df

569it [00:20, 28.44it/s]


,account,id_str
0,1002630999052865536,1477015231444434952
1,1002630999052865536,1462898232367931410
2,1002630999052865536,702826262989934592
3,1002630999052865536,1646872583474761730
4,1002630999052865536,1645861187836432401
...,...,...
2831703,996094929733652481,1614838640932450304
2831704,996094929733652481,1534857849037328386
2831705,996094929733652481,2994059088
2831706,996094929733652481,1534043955708542977


In [111]:
followers_df = pd.read_csv("./460_account_followers.csv")
# followers_df = pf_df

In [112]:
str_user_ids = set(map(str, user_ids)) 

In [113]:
followers_df

,account,id_str
0,RoyBlunt,1434201858864726019
1,RoyBlunt,1469657120283009025
2,RoyBlunt,1373365280508809219
3,RoyBlunt,1262788067057979395
4,RoyBlunt,1470943571180150787
...,...,...
22230396,RepAnthonyBrown,713094905552248832
22230397,RepAnthonyBrown,1408858718
22230398,RepAnthonyBrown,19650150
22230399,RepAnthonyBrown,22781325


In [114]:
'1477015231444434952' in accounts_who_follow_poli

True

In [115]:
accounts_who_follow_poli = set(followers_df.id_str)

In [116]:
len(accounts_who_follow_poli)

6993089

In [117]:
accounts_who_follow_poli[0]

TypeError: 'set' object does not support indexing

In [ ]:
non_poli_users = []
poli_users = []

for user_id in tqdm(str_user_ids):
    if user_id in accounts_who_follow_poli:
        poli_users.append(user_id)
    else:
        non_poli_users.append(user_id)
len(non_poli_users)

In [ ]:
users_with_estimate = str_user_ids.intersection(accounts_who_follow_poli)

In [ ]:
len(users_with_estimate)

In [ ]:
filtered_followers_df = followers_df[followers_df.id_str.isin(user_ids)]

In [ ]:
len(set(filtered_followers_df.id_str))

In [ ]:
unique_user_id_df = pd.DataFrame({"id_str":list(user_ids)})

In [118]:
# unique_user_id_df = unique_user_id_df.rename(columns={"user.id_str":"id_str"})
join_df = pd.merge(followers_df, unique_user_id_df, how="inner", on="id_str")

In [119]:
join_df

,account,id_str
0,RoyBlunt,20230439
1,MartinHeinrich,20230439
2,RepTimBurchett,20230439
3,SenRubioPress,20230439
4,RepRubenGallego,20230439
...,...,...
2935218,RepAnthonyBrown,224703460
2935219,RepAnthonyBrown,553804155
2935220,RepAnthonyBrown,3180852639
2935221,RepAnthonyBrown,451776738


In [120]:
adj_matrix = pd.DataFrame(0, index=join_df["id_str"].unique(), columns=join_df["account"].unique())

In [121]:
adj_matrix

,RoyBlunt,MartinHeinrich,RepTimBurchett,SenRubioPress,RepRubenGallego,gillibrandny,RepCori,RepStephMurphy,RepGregSteube,RepMaryMiller,...,RepHorsford,repdinatitus,RepKweisiMfume,Kilili_Sablan,RepAmata,RepAnnieKuster,GuamCongressman,EconUS,SenMurphyOffice,BlumenauerMedia
20230439,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
906055412092473344,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15042333,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1011374689053130752,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
388893206,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224703460,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
553804155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3180852639,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
451776738,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [124]:
for poli_account in tqdm(join_df["account"].unique()):
    followers_in_users = join_df[join_df["account"]==poli_account]["id_str"]
    adj_matrix[poli_account][followers_in_users]=1

100%|██████████| 460/460 [00:54<00:00,  8.40it/s]


In [125]:
adj_matrix

,RoyBlunt,MartinHeinrich,RepTimBurchett,SenRubioPress,RepRubenGallego,gillibrandny,RepCori,RepStephMurphy,RepGregSteube,RepMaryMiller,...,RepHorsford,repdinatitus,RepKweisiMfume,Kilili_Sablan,RepAmata,RepAnnieKuster,GuamCongressman,EconUS,SenMurphyOffice,BlumenauerMedia
20230439,1,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
906055412092473344,1,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
15042333,1,0,1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1011374689053130752,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
388893206,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224703460,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
553804155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3180852639,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
451776738,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [126]:
adj_matrix.to_csv("./Barbera Political Ideology/poli_account_followers/adj_jason_460.csv")

In [ ]:
ad